In [1]:
import warnings
warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

import tensorflow as tf

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
%matplotlib inline

import tensorflow_hub as thub
import tensorflow.keras.backend as K

from lib.data_utils import *

In [99]:
from art.classifiers import KerasClassifier
from art.attacks import FastGradientMethod
from art.attacks.extraction import CopycatCNN
import art

In [207]:
def sample_by_class(data, labels, num_samples=100):
    sample_data = []
    sample_labels = []
    unq_labels = list(range(labels.shape[1]))
    for label in unq_labels:
        idx = labels[:,label]==1
        sample_set = data[idx][0:num_samples].copy()
        label_set = labels[idx][0:num_samples].copy()
        sample_data.append(sample_set)
        sample_labels.append(label_set)
    
    sample_data = np.concatenate(sample_data)
    sample_labels = np.concatenate(sample_labels)
    print(sample_data.shape, sample_labels.shape)
    return sample_data, sample_labels

In [3]:
# from tensorflow.keras.models import Model
# from tensorflow.keras.layers import Input
# from tensorflow.keras import Sequential
# from tensorflow.keras.optimizers import SGD
# from tensorflow.keras.losses import CategoricalCrossentropy
# from tensorflow.keras.utils import to_categorical


In [4]:
# Load the raw CIFAR-10 data
cifar10_dir = 'lib/datasets/cifar-10-batches-py'
x_train, y_train, x_test, y_test = load_cifar10(cifar10_dir)

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Convert class vectors to binary class matrices.
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [5]:
testd = x_train[0:1000].copy()
testl = y_train[0:1000].copy()

In [84]:
K.clear_session()
model_url = "https://tfhub.dev/deepmind/ganeval-cifar10-convnet/1"

ganeval_module = thub.Module(model_url)
# gan_eval_layer = thub.KerasLayer(model_url)

In [85]:
ge_cifar_clf = tf.keras.Sequential()
ge_cifar_clf.add(tf.keras.layers.InputLayer(input_shape=(32,32,3)))
# gan_eval_model.add(gan_eval_layer)
ge_cifar_clf.add(thub.KerasLayer(ganeval_module))
ge_cifar_clf.add(tf.keras.layers.Activation('softmax'))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [86]:
ge_cifar_clf.compile(optimizer=tf.keras.optimizers.SGD(lr=0.1, momentum=0.9, decay=1e-4),
                       loss=tf.keras.losses.CategoricalCrossentropy(),
                       metrics=['accuracy'])
ge_cifar_clf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 10)                7796426   
_________________________________________________________________
activation (Activation)      (None, 10)                0         
Total params: 7,796,426
Trainable params: 0
Non-trainable params: 7,796,426
_________________________________________________________________


### Testing the retrieved hub model by evaluating on train data
Should give us 100% or something close to that

In [13]:
ge_cifar_clf.evaluate(testd, testl)

1000/1000 [==============================] - 3s 3ms/sample - loss: 0.0017 - acc: 1.0000


[0.001668029203079641, 1.0]

### Testing a sample attack

In [10]:
classifier = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)
attack_fgsm = FastGradientMethod(classifier=classifier, eps=0.4)

In [11]:
x_test_adv = attack_fgsm.generate(testd.copy())

In [12]:
ge_cifar_clf.evaluate(x_test_adv, testl)

1000/1000 [==============================] - 3s 3ms/sample - loss: 6.9334 - acc: 0.1020


[6.933382461547851, 0.102]

### Define the ART classifier for our model

In [ ]:
from art.classifiers import Classifier, BlackBoxClassifier
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [ ]:
def predict(x):
    out = np.argmax(ge_cifar_clf.predict(x.reshape(-1,32,32,3)),axis=1)
    return to_categorical(out, 10)

In [ ]:
sub_clf = tf.keras.Sequential( )
sub_clf.add( Conv2D( 32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32,32,3) ) )
sub_clf.add( Conv2D( 64, (3, 3), padding='same', activation='relu' ) )
sub_clf.add( MaxPooling2D( pool_size=(2, 2) ) )
sub_clf.add( Dropout( 0.25 ) )
sub_clf.add( Flatten( ) )
sub_clf.add( Dense( 128, activation='relu' ) )
# sub_clf.add( Dropout( 0.5 ) )
sub_clf.add( Dense(10, activation='softmax' ) )

sub_clf.compile( loss=tf.keras.losses.categorical_crossentropy,
                 optimizer='adam',
                 metrics=[ 'accuracy' ] )

sub_clf.summary( )

In [ ]:
x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=1000)

In [ ]:
art_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)
extracted_art_clf = KerasClassifier(model=sub_clf, clip_values=(0, 1), use_logits=False)

In [ ]:
attack_CCC = CopycatCNN(classifier=art_clf, 
                        batch_size_fit=8, 
                        batch_size_query=8,
                        nb_epochs=20,
                        nb_stolen=1000)

In [ ]:
attack_CCC.extract(x_train_adv, y_train_adv, thieved_classifier=extracted_art_clf)

In [ ]:
extracted_art_clf._model.evaluate(x_test, y_test)

In [191]:
from art.classifiers import Classifier, BlackBoxClassifier
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

In [192]:
def predict(x):
    out = np.argmax(ge_cifar_clf.predict(x.reshape(-1,32,32,3)),axis=1)
    return to_categorical(out, 10)

In [196]:
sub_clf = tf.keras.Sequential( )
sub_clf.add( Conv2D( 32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(32,32,3) ) )
sub_clf.add( Conv2D( 64, (3, 3), padding='same', activation='relu' ) )
sub_clf.add( MaxPooling2D( pool_size=(2, 2) ) )
sub_clf.add( Dropout( 0.25 ) )
sub_clf.add( Flatten( ) )
sub_clf.add( Dense( 128, activation='relu' ) )
# sub_clf.add( Dropout( 0.5 ) )
sub_clf.add( Dense(10, activation='softmax' ) )

sub_clf.compile( loss=tf.keras.losses.categorical_crossentropy,
                 optimizer='adam',
                 metrics=[ 'accuracy' ] )

sub_clf.summary( )

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 16384)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               2097280   
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)              

In [208]:
x_train_adv, y_train_adv = sample_by_class(x_train, y_train, num_samples=1000)

(10000, 32, 32, 3) (10000, 10)


In [209]:
art_clf = KerasClassifier(model=ge_cifar_clf, clip_values=(0, 1), use_logits=False)
extracted_art_clf = KerasClassifier(model=sub_clf, clip_values=(0, 1), use_logits=False)

In [210]:
attack_CCC = CopycatCNN(classifier=art_clf, 
                        batch_size_fit=8, 
                        batch_size_query=8,
                        nb_epochs=20,
                        nb_stolen=1000)

In [211]:
attack_CCC.extract(x_train_adv, y_train_adv, thieved_classifier=extracted_art_clf)

Epoch 1/20
125/125 [==============================] - 3s 20ms/step - loss: 1.9875 - acc: 0.3050
Epoch 2/20
125/125 [==============================] - 2s 19ms/step - loss: 1.7236 - acc: 0.3600
Epoch 3/20
125/125 [==============================] - 2s 20ms/step - loss: 1.5235 - acc: 0.4600
Epoch 4/20
125/125 [==============================] - 3s 23ms/step - loss: 1.3009 - acc: 0.5450
Epoch 5/20
125/125 [==============================] - 3s 22ms/step - loss: 1.0874 - acc: 0.6110
Epoch 6/20
125/125 [==============================] - 3s 20ms/step - loss: 1.0082 - acc: 0.6600
Epoch 7/20
125/125 [==============================] - 3s 21ms/step - loss: 0.7881 - acc: 0.7230
Epoch 8/20
125/125 [==============================] - 3s 21ms/step - loss: 0.7183 - acc: 0.7400
Epoch 9/20
125/125 [==============================] - 3s 20ms/step - loss: 0.6435 - acc: 0.7870
Epoch 10/20
125/125 [==============================] - 3s 21ms/step - loss: 0.4904 - acc: 0.8390
Epoch 11/20
125/125 [==================

art.classifiers.keras.KerasClassifier(model=<tensorflow.python.keras.engine.sequential.Sequential object at 0x141f884a8>, use_logits=False, channel_index=3, clip_values=(0, 1), preprocessing_defences=None, postprocessing_defences=None, preprocessing=(0, 1), input_layer=0, output_layer=0)

In [212]:
extracted_art_clf._model.evaluate(x_test, y_test)

10000/10000 [==============================] - 3s 304us/sample - loss: 2.7106 - acc: 0.4243


[2.7106353725433348, 0.4243]